---
title: Results
format:
  html:
    code-fold: true
output-file: overview.html
---

In [ ]:
from beforerr.r import py2rpy_polars

In [ ]:
#| hide
%load_ext autoreload
%autoreload 2

## Setup

### Import libraries

In [ ]:
from fastcore.utils import *
from fastcore.test import *

from ids_finder.utils.basic import *
from ids_finder.core import pl_norm

from datetime import timedelta

import polars as pl
import polars.selectors as cs
import pandas as pd
import numpy as np

import seaborn as sns
import altair as alt
import vegafusion as vf
vf.enable(row_limit = int(1e6))

vegafusion.enable(mimetype='html', row_limit=1000000, embed_options=None)

## Visualization of data gaps

## JUNO Overview
### Magnetic field strength vs. distance

## Classification of candidates

In [ ]:
sns.jointplot(
    candidates_jno_tau_60s,
    x='dBOverB', y='BnOverB',
    # kind='kde',
    kind="hex",
)

In [ ]:
sns.jointplot(
    all_candidates,
    x='dBOverB', y='BnOverB',
    # kind='kde',
    kind="hex",
)

## Distribution of the DD types

##### Plot distribution of types for each missions


In [ ]:
alt.Chart(all_candidates).encode(
    x=alt.X("count()").stack("normalize").title("Share of ID types"),
    y=alt.Y('sat').title(None),
    color='type',
).mark_bar()

# alt.Chart(distributions).encode(
#     alt.X('ratio:Q', title='DD type distribution').axis(format='.0%'),
#     y=alt.Y('sat', title=None),
#     color='type',
# ).mark_bar()

In [ ]:
distributions = all_candidates.group_by("sat", "type").agg(pl.count()).with_columns(
    (pl.col("count") / pl.sum("count").over("sat")).alias("ratio")
)
count_table = distributions.to_pandas().pivot(index='sat', columns='type', values='count')[['RD', 'TD', 'ED', 'ND']]
ratio_table = distributions.to_pandas().pivot(index='sat', columns='type', values='ratio')[['RD', 'TD', 'ED', 'ND']]
# display(distributions.to_pandas().pivot(index='sat', columns='type', values='ratio')[['RD', 'TD', 'ED', 'ND']].style.format("{:.0%}"))
display(count_table, ratio_table.style.format("{:.0%}"))


type,RD,TD,ED,ND
sat,,,,
JNO,34003,1576,15595,1277
STA,71337,2905,29916,1727
THB,26793,1093,8359,1193


type,RD,TD,ED,ND
sat,,,,
JNO,65%,3%,30%,2%
STA,67%,3%,28%,2%
THB,72%,3%,22%,3%


##### Plot distribution of types for each missions **over time**

In [ ]:
alt.Chart(all_candidates).mark_bar(binSpacing=0).encode(
    x="yearmonth(time)",
    y=alt.Y("count()").stack("normalize").title("Share of ID types"),
    row=alt.Row("sat").title(None),
    color="type",
).configure_axis(grid=False).properties(height=100)

alt.Chart(...)

In [ ]:
%%R
plot_type_distribution <- function(data, bin_width = 30) { 
    data$date_only <- as.Date(data$time)
    
    p <- ggplot(data, aes(date_only, fill = type)) +
        geom_histogram(binwidth = bin_width, position = "fill") + 
        theme_pubr(base_size = 16)
        
    return(p)
}

p1 <- plot_type_distribution(jno_candidates)
p2 <- plot_type_distribution(thb_candidates)
p3 <- plot_type_distribution(sta_candidates)

p <- ggarrange(
    p1 + rremove("xlab"),
    p2 + rremove("xlab"), p3, 
    nrow = 3, align = 'hv', 
    labels=list("JUNO", "ARTEMIS-B", "STEREO-A"), hjust=0, vjust=0,
    legend = 'right', common.legend = TRUE
)
# save_plot(filename = "type_distribution")
p

### $dB_l/B$

- $dB_l/B$ does not show any clear correlation with time.

### Waiting time

### Amplitude

### Plotting candidates of different types of discontinuities

In [ ]:
# alt.Chart(candidates).mark_point().encode(
#     x='yearmonth(time)',
#     y='count()',
#     color='type',
# )

In [ ]:
alt.Chart(candidates_jno_tau_60s.to_pandas()).mark_point().encode(
    x='X',
    y='count(type)',
    color='type',
)

### Duration